# 0 - Inicialização

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from plotly import graph_objects as go
import plotly as py

from datetime import datetime
from datetime import timedelta

import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix, accuracy_score, f1_score, confusion_matrix, recall_score, precision_score

from sklearn.utils import resample

# 1 - Classificar o dataset clusterizado por dia

## Carregar dados

In [2]:
df_cluster = pd.read_csv('../../../data/cleandata/Info pluviometricas/Merged Data/clustered_label.csv', sep = ';')
df_cluster['Data_Hora'] = pd.to_datetime(df_cluster['Data_Hora'], yearfirst=True)
df_cluster['Data'] = df_cluster['Data_Hora'].dt.strftime('%Y-%m-%d')
df_cluster

,Data_Hora,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,LocalMax_Dia,LocalMax_Hora,Data
0,2011-01-01 00:00:00,Erasmo,88.8,920.5,10.243265,22.9,17.700000,0.000000,133.0,0.7,0.0,0,0,2011-01-01
1,2011-01-01 00:00:00,Camilopolis,84.8,920.6,20.200000,22.8,17.500000,0.000000,137.0,1.5,0.0,0,0,2011-01-01
2,2011-01-01 00:00:00,Paraiso,92.3,919.1,19.300000,22.7,18.000000,0.000000,101.0,2.6,0.0,0,0,2011-01-01
3,2011-01-01 00:00:00,Vitoria,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0,0,2011-01-01
4,2011-01-01 00:00:00,RM,86.6,926.7,20.100000,23.3,17.800000,0.000000,131.0,2.5,0.0,0,0,2011-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463565,2019-06-30 23:45:00,Camilopolis,54.4,924.0,20.800000,23.0,12.200000,0.073893,332.0,4.3,0.0,0,0,2019-06-30
1463566,2019-06-30 23:45:00,RM,66.9,931.9,18.800000,22.5,13.300000,0.294461,331.0,0.7,0.0,0,0,2019-06-30
1463567,2019-06-30 23:45:00,Vitoria,38.4,925.8,20.300000,27.5,6.300000,0.000000,0.0,0.0,0.0,0,0,2019-06-30
1463568,2019-06-30 23:45:00,Paraiso,55.3,924.2,20.800000,22.1,11.790525,0.000000,25.0,3.6,0.0,0,0,2019-06-30


## Agrupar por dia

In [12]:
# df_cluster_dia = df_cluster.drop(columns = ['Data_Hora', 'LocalMax_Hora']).rename(columns = {'LocalMax_Dia': 'LocalMax'})
# s_prec_p = df_cluster_dia.groupby(['Data', 'Local']).sum().reset_index().groupby('Data').mean()['Precipitacao'].reset_index()
# s_prec_o = df_cluster_dia.groupby(['Data', 'Local']).max().reset_index().groupby('Data').max()['LocalMax'].reset_index()
# df_cluster_dia = df_cluster_dia.groupby(['Data']).mean().reset_index()
# df_cluster_dia['Precipitacao'] = s_prec_p['Precipitacao']
# df_cluster_dia['LocalMax'] = s_prec_o['LocalMax']
# df_cluster_dia

df_cluster_dia = df_cluster.drop(columns = ['Data_Hora', 'LocalMax_Hora']).rename(columns = {'LocalMax_Dia': 'LocalMax'})
df_prec_p = df_cluster_dia.groupby(['Data', 'Local']).sum()['Precipitacao'].reset_index()
df_prec_o = df_cluster_dia.groupby(['Data', 'Local']).max()['LocalMax'].reset_index()
df_cluster_dia = df_cluster_dia.groupby(['Data', 'Local']).mean().reset_index().drop(columns = ['Precipitacao', 'LocalMax'])
df_cluster_dia = df_cluster_dia.merge(df_prec_p, on = ['Data', 'Local'])
df_cluster_dia = df_cluster_dia.merge(df_prec_o, on = ['Data', 'Local'])
df_cluster_dia['Local'] = df_cluster_dia['Local'].replace({'Camilopolis': 1, 'Erasmo': 2, 'Paraiso': 3, 'RM': 4, 'Vitoria': 5})
df_cluster_dia['Mes'] = df_cluster_dia['Data'].str[5:7].astype(int)
df_cluster_dia

,Data,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,LocalMax,Mes
0,2011-01-01,1,80.037996,918.279493,20.981523,24.507206,17.288055,132.072917,143.062500,2.160417,0.6,0,1
1,2011-01-01,2,82.882941,918.304598,20.023345,24.302490,17.296642,134.354167,124.833333,2.959375,2.2,0,1
2,2011-01-01,3,83.588051,916.817234,20.256149,24.174391,17.407144,160.770833,146.114583,2.661458,1.6,0,1
3,2011-01-01,4,81.887094,924.393186,20.809716,24.522501,17.479821,154.968750,158.187500,2.942708,3.2,0,1
4,2011-01-01,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15345,2019-06-30,1,50.984375,924.260417,21.940625,24.225000,12.049696,146.491123,301.906250,13.586458,0.0,0,6
15346,2019-06-30,2,52.906250,925.870833,21.695833,24.270707,11.334375,0.000000,237.510417,8.039442,0.0,0,6
15347,2019-06-30,3,52.266667,924.503125,22.107292,23.691667,11.917331,142.864583,144.958333,9.913450,0.0,0,6
15348,2019-06-30,4,54.458333,932.079167,21.626042,25.279167,12.515625,169.933156,263.375000,6.782296,0.0,0,6


In [13]:
xgb = xgboost.XGBClassifier()

cols_rem = ['LocalMax', 'Data']

x = df_cluster_dia[[c for c in df_cluster_dia.columns if c not in cols_rem]]
y = df_cluster_dia['LocalMax']

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state = 378)

# concatenate our training data back together
X = pd.concat([x_treino, y_treino], axis=1)

# separate minority and majority classes
not_ordem = X[X['LocalMax']==0].copy()
ordem = X[X['LocalMax']==1].copy()

# upsample minority
ordem_upsampled = resample(ordem,
                        replace=True, # sample with replacement
                        n_samples=len(not_ordem), # match number in majority class
                        random_state=378) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_ordem, ordem_upsampled])

x_treino = upsampled[[c for c in df_cluster_dia.columns if c not in cols_rem]]
y_treino = upsampled['LocalMax']

display(y_treino.value_counts())

#xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino), (x_teste, y_teste)], eval_metric=f1_score)
param = {'max_depth':10, 'eta':2, 'objective':'binary:logistic', 'min_child_weight': 1, 'lambda': 1, 'alpha': 0, 'gamma': 0}

df_train = xgboost.DMatrix(data=x_treino, label=y_treino)

bst = xgboost.train(param, df_train, 2, feval=f1_score)
y_teste_pred = bst.predict(xgboost.DMatrix(data=x_teste, label=y_teste))
y_teste_pred = [1 if i>0.5 else 0 for i in y_teste_pred]
y_treino_pred = bst.predict(xgboost.DMatrix(data=x_treino, label=y_treino))
y_treino_pred = [1 if i>0.5 else 0 for i in y_treino_pred]

print(f"Treino: {accuracy_score(y_treino, y_treino_pred)}")
print(f"Teste: {accuracy_score(y_teste, y_teste_pred)}")
print(f"Precisão: {precision_score(y_teste, y_teste_pred)}")
print(f"Recall: {recall_score(y_teste, y_teste_pred)}")
print(f"F1: {f1_score(y_teste, y_teste_pred)}")
display(confusion_matrix(y_teste, y_teste_pred, normalize='true'))
display(confusion_matrix(y_teste, y_teste_pred,))

1    10448
0    10448
Name: LocalMax, dtype: int64

Treino: 0.9100784839203675
Teste: 0.9155266015200869
Precisão: 0.0804953560371517
Recall: 0.22033898305084745
F1: 0.11791383219954647


array([[0.93380878, 0.06619122],
       [0.77966102, 0.22033898]])

array([[4190,  297],
       [  92,   26]], dtype=int64)

# 2 - Classificar o dataset clusterizado por 15 mins

In [104]:
df_label_d = pd.read_csv('../../../data/cleandata/Ordens de serviço/labels_day.csv', sep = ';')
df_cluster_hora = df_cluster.merge(df_label_d[['Data', 'LocalMax']], on = 'Data', how = 'left').fillna(0)
df_cluster_hora = df_cluster_hora.drop(columns=['LocalMax_Hora', 'LocalMax_Dia'])

In [105]:
#df_cluster_hora = df_cluster.copy()
#df_cluster_hora = df_cluster_hora.drop(columns=['LocalMax_Hora']).rename(columns = {'LocalMax_Dia': 'LocalMax'})
df_cluster_hora = df_cluster_hora.sort_values(by=['Data_Hora', 'Local'])

In [106]:
df_cluster_hora.groupby('LocalMax').count()

,Data_Hora,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Data
LocalMax,,,,,,,,,,,,
0.0,1418705,1418705,1418705,1418705,1418705,1418705,1418705,1418705,1418705,1418705,1418705,1418705
1.0,44865,44865,44865,44865,44865,44865,44865,44865,44865,44865,44865,44865


Prec24h -> soma da precipitação nas últimas 24 horas

In [107]:
# for i in range(24*4):
#     df_cluster_hora['Prec_Shift_' + str(i)] = df_cluster_hora['Precipitacao'].shift(-5 * (i+1), fill_value=0)

# df_cluster_hora['Prec24h'] = df_cluster_hora[[c for c in df_cluster_hora.columns if 'Prec_Shift' in c]].sum(axis=1)
# df_cluster_hora = df_cluster_hora.drop(columns = [c for c in df_cluster_hora.columns if 'Prec_Shift' in c])
# df_cluster_hora.head(10)

PrecMomento -> soma da precipitação até aquele momento do dia

In [127]:
init_time = datetime.now()
def getPrecMomento(x):
    return df_cluster_hora.loc[
        (df_cluster_hora['Data'] == x['Data']) &
        (df_cluster_hora['Data_Hora'] <= x['Data_Hora']) &
        (df_cluster_hora['Local'] == x['Local']), 'Precipitacao'
    ].sum()

#df_cluster_hora['PrecMomento'] = df_cluster_hora.apply(getPrecMomento, axis=1)
df_cluster_hora.head(100).apply(getPrecMomento, axis=1)
print(datetime.now() - init_time)

0:00:14.958399


In [128]:
df_cluster_hora.shape

(1463570, 13)

PrecSum -> soma da precipitação no dia

In [92]:
df_prec_sum = df_cluster_hora.groupby(['Data', 'Local']).sum().reset_index()[['Data', 'Local', 'Precipitacao']]
df_prec_sum.columns = ['Data', 'Local', 'PrecSum']
df_cluster_hora = df_cluster_hora.merge(df_prec_sum, on=['Data', 'Local'])
df_cluster_hora.head(10)

,Data_Hora,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Data,LocalMax,Prec24h,PrecSum
0,2011-01-01 00:00:00,Camilopolis,84.8,920.6,20.2,22.800000,17.5,0.0,137.0,1.5,0.0,2011-01-01,0.0,0.8,0.6
1,2011-01-01 00:15:00,Camilopolis,84.1,920.3,20.1,22.800000,17.4,0.0,76.0,1.9,0.0,2011-01-01,0.0,0.8,0.6
2,2011-01-01 00:30:00,Camilopolis,83.9,920.2,20.1,22.900000,17.2,0.0,92.0,2.9,0.0,2011-01-01,0.0,0.8,0.6
3,2011-01-01 00:45:00,Camilopolis,83.5,919.9,20.2,22.900000,17.3,0.0,133.0,1.4,0.0,2011-01-01,0.0,0.8,0.6
4,2011-01-01 01:00:00,Camilopolis,84.0,919.9,20.1,22.800000,17.3,0.0,151.0,1.8,0.0,2011-01-01,0.0,0.8,0.6
5,2011-01-01 01:15:00,Camilopolis,84.5,919.5,20.0,23.143578,17.3,0.0,124.0,1.8,0.0,2011-01-01,0.0,0.8,0.6
6,2011-01-01 01:30:00,Camilopolis,84.6,919.4,20.0,23.083792,17.3,0.0,137.0,1.8,0.0,2011-01-01,0.0,0.8,0.6
7,2011-01-01 01:45:00,Camilopolis,84.5,919.4,20.0,23.005875,17.3,0.0,115.0,1.9,0.0,2011-01-01,0.0,1.0,0.6
8,2011-01-01 02:00:00,Camilopolis,85.8,919.2,19.8,22.935631,17.4,0.0,155.0,1.8,0.0,2011-01-01,0.0,1.2,0.6
9,2011-01-01 02:15:00,Camilopolis,86.5,919.0,19.7,22.786449,17.4,0.0,140.0,1.7,0.0,2011-01-01,0.0,1.6,0.6


In [29]:
# df_clustered_total['Hora'] = pd.to_datetime(df_clustered_total['Data_Hora'], yearfirst=True).dt.hour

# df_ohe = df_clustered_total.groupby(['Data', 'Local', 'Hora']).sum().reset_index()[['Data', 'Local', 'Hora', 'Precipitacao']]
# s_ohe = df_ohe['Hora']
# df_ohe = pd.get_dummies(df_ohe, columns = ['Hora'])
# df_ohe['Hora'] = s_ohe

# for i in range(24):
#     df_ohe.loc[df_ohe['Hora_' + str(i)] == 1, 'Hora_' + str(i)] = df_ohe.loc[df_ohe['Hora_' + str(i)] == 1, 'Precipitacao']

# df_clustered_total = df_clustered_total.merge(df_ohe[['Data', 'Local'] + [c for c in df_ohe.columns if 'Hora' in c]], on=['Data', 'Local', 'Hora'])

In [93]:
df_cluster_hora_a = df_cluster_hora.copy()

In [94]:
df_cluster_hora['Hora'] = df_cluster_hora['Data_Hora'].dt.hour
df_cluster_hora['Mes'] = df_cluster_hora['Data_Hora'].dt.month
df_cluster_hora['Local'] = df_cluster_hora['Local'].replace({'Camilopolis': 1, 'Erasmo': 2, 'Paraiso': 3, 'RM': 4, 'Vitoria': 5})


# Filtrando hora no minuto 0
df_hora = df_cluster_hora.groupby(['Data', 'Local', 'Hora']).sum().reset_index()[['Data', 'Local', 'Hora', 'Precipitacao']]
# df_clustered_total = df_clustered_total.groupby(['Data', 'Local', 'Hora']).mean().reset_index()
# s_prec = df_clustered_total.groupby(['Data', 'Local', 'Hora']).sum()[['Precipitacao']]
# df_clustered_total['Precipitacao'] = s_prec

df_cluster_hora['Minuto'] = df_cluster_hora['Data_Hora'].dt.minute
df_cluster_hora = df_cluster_hora[df_cluster_hora['Minuto'] == 0]
#df_cluster_hora = df_cluster_hora.drop(columns = ['Data_Hora', 'Minuto'])
# df_clustered_total = df_clustered_total.drop(columns = ['Data_Hora', 'Minuto', 'Precipitacao'])
# df_clustered_total = df_clustered_total.merge(df_hora, on=['Data', 'Local', 'Hora'])

In [95]:
#df_slice = df_clustered_total[(df_clustered_total['Ordens'] == 1) | (df_clustered_total['Cluster'].isin([1,2]))]
#df_slice = df_clustered_total[(df_clustered_total['Ordens'] == 1) | (df_clustered_total['PrecSum'] > 10)]
df_slice = df_cluster_hora.copy()
#df_slice = df_clustered_total[(df_clustered_total['Cluster'].isin([0]))]
#df_slice.loc[df_slice['Cluster'] == 0, 'Ordens'] = 0

df_slice.loc[(df_slice['LocalMax'] == 1) & (df_slice['PrecSum'] <= 10), 'LocalMax'] = 0

#df_slice.loc[(df_slice['Ordens'] == 1) & ~((df_clustered_total[[c for c in df_clustered_total.columns if 'Hora_' in c]] <= 20).sum(axis = 1) < 24), 'Ordens'] = 0
#df_slice = df_slice[df_slice['Local'] == 4]

In [96]:
df_slice.groupby('LocalMax').count()

,Data_Hora,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Data,Prec24h,PrecSum,Hora,Mes,Minuto
LocalMax,,,,,,,,,,,,,,,,,
0.0,358978,358978,358978,358978,358978,358978,358978,358978,358978,358978,358978,358978,358978,358978,358978,358978,358978
1.0,6952,6952,6952,6952,6952,6952,6952,6952,6952,6952,6952,6952,6952,6952,6952,6952,6952


In [97]:
df_slice.shape

(365930, 18)

In [84]:
for l in range(6):
    if l != 0:
        df_train = df_slice[df_slice['Local'] == l]
    else:
        df_train = df_slice.copy()
        
    print(f'----- LOCAL {l} -----')

    # Testar prever cluster

    xgb = xgboost.XGBClassifier()

    cols_rem = ['LocalMax', 'Cluster', 'Data', 'Hora', 'Data_Hora', 'Ordens', 'Minuto'] + [c for c in df_train.columns if 'Hora_' in c]
    #cols_rem = cols_rem + ['DirecaoDoVento', 'VelocidadeDoVento']
    #cols_rem = cols_rem + ['TemperaturaInterna', 'PontoDeOrvalho']

    x = df_train[[c for c in df_train.columns if c not in cols_rem]]
    if l == 0:
        print(x.columns)
    #x = x.drop(columns = 'Cluster')
    y = df_train['LocalMax']

    x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state = 378, stratify=y)

    # concatenate our training data back together
    X = pd.concat([x_treino, y_treino], axis=1)

    # separate minority and majority classes
    not_ordem = X[X['LocalMax']==0].copy()
    ordem = X[X['LocalMax']==1].copy()

    # upsample minority
    ordem_upsampled = resample(ordem,
                            replace=True, # sample with replacement
                            n_samples=len(not_ordem), # match number in majority class
                            random_state=378) # reproducible results

    # combine majority and upsampled minority
    upsampled = pd.concat([not_ordem, ordem_upsampled])

    x_treino = upsampled[[c for c in df_slice.columns if c not in cols_rem]]
    y_treino = upsampled['LocalMax']

    display(y_treino.value_counts())

    #xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino), (x_teste, y_teste)], eval_metric=f1_score)
    param = {'max_depth':50, 'eta':1, 'objective':'binary:logistic', 'min_child_weight': 1, 'lambda': 1, 'alpha': 0, 'gamma': 0}

    df_train = xgboost.DMatrix(data=x_treino, label=y_treino)
    df_test = xgboost.DMatrix(data=x_teste, label=y_teste)

    bst = xgboost.train(param, df_train, 2, feval=f1_score)
    y_teste_pred = bst.predict(xgboost.DMatrix(data=x_teste, label=y_teste))
    y_teste_pred = [1 if i>0.5 else 0 for i in y_teste_pred]
    y_treino_pred = bst.predict(xgboost.DMatrix(data=x_treino, label=y_treino))
    y_treino_pred = [1 if i>0.5 else 0 for i in y_treino_pred]

    print(f"Treino: {accuracy_score(y_treino, y_treino_pred)}")
    print(f"Teste: {accuracy_score(y_teste, y_teste_pred)}")
    print(f"Precisão: {precision_score(y_teste, y_teste_pred)}")
    print(f"Recall: {recall_score(y_teste, y_teste_pred)}")
    print(f"F1: {f1_score(y_teste, y_teste_pred)}")
    display(confusion_matrix(y_teste, y_teste_pred, normalize='true'))
    display(confusion_matrix(y_teste, y_teste_pred,))

----- LOCAL 0 -----
Index(['Local', 'UmidadeRelativa', 'PressaoAtmosferica', 'TemperaturaDoAr',
       'TemperaturaInterna', 'PontoDeOrvalho', 'RadiacaoSolar',
       'DirecaoDoVento', 'VelocidadeDoVento', 'Precipitacao', 'PrecSum',
       'Mes'],
      dtype='object')


1.0    251866
0.0    251866
Name: LocalMax, dtype: int64

Treino: 0.9954003319225302
Teste: 0.9758697018555461
Precisão: 0.36009648518263265
Recall: 0.5688622754491018
F1: 0.4410213125131885


array([[0.98279632, 0.01720368],
       [0.43113772, 0.56886228]])

array([[106085,   1857],
       [   792,   1045]], dtype=int64)

...

----- LOCAL 5 -----


1.0    50477
0.0    50477
Name: LocalMax, dtype: int64

Treino: 0.9961368544089387
Teste: 0.9800965567498634
Precisão: 0.3903846153846154
Recall: 0.628482972136223
F1: 0.4816132858837485


array([[0.98534646, 0.01465354],
       [0.37151703, 0.62848297]])

array([[21316,   317],
       [  120,   203]], dtype=int64)

In [98]:
test_cases = [
    [],
    ['DirecaoDoVento', 'VelocidadeDoVento'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna', 'PontoDeOrvalho'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna', 'PontoDeOrvalho', 'UmidadeRelativa'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna', 'PontoDeOrvalho', 'UmidadeRelativa', 'PressaoAtmosferica'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna', 'PontoDeOrvalho', 'UmidadeRelativa', 'PressaoAtmosferica', 'RadiacaoSolar'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna', 'PontoDeOrvalho', 'UmidadeRelativa', 'PressaoAtmosferica', 'RadiacaoSolar', 'TemperaturaDoAr'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna', 'PontoDeOrvalho', 'UmidadeRelativa', 'PressaoAtmosferica', 'RadiacaoSolar', 'TemperaturaDoAr', 'Local'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna', 'PontoDeOrvalho', 'UmidadeRelativa', 'PressaoAtmosferica', 'RadiacaoSolar', 'TemperaturaDoAr', 'PrecSum'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna', 'PontoDeOrvalho', 'UmidadeRelativa', 'PressaoAtmosferica', 'RadiacaoSolar', 'TemperaturaDoAr', 'Precipitacao']
]

df_training_result = pd.DataFrame(columns = ['Removed_Cols', 'Remaining_Cols', 'Local', 'Train_Acc', 'Test_Acc', 'Precision', 'Recall', 'F1', 'Ver_Pos'])

for case in test_cases:
    for l in range(6):
        if l != 0:
            df_train = df_slice[df_slice['Local'] == l]
        else:
            df_train = df_slice.copy()

        print(f'----- LOCAL {l} -----')

        # Testar prever cluster

        xgb = xgboost.XGBClassifier()

        cols_rem = ['LocalMax', 'Cluster', 'Data', 'Hora', 'Data_Hora', 'Ordens', 'Minuto'] + [c for c in df_train.columns if 'Hora_' in c]
        cols_rem = cols_rem + case
        #cols_rem = cols_rem + ['TemperaturaInterna', 'PontoDeOrvalho']

        x = df_train[[c for c in df_train.columns if c not in cols_rem]]
        if l == 0:
            print(x.columns)
        #x = x.drop(columns = 'Cluster')
        y = df_train['LocalMax']

        x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state = 378, stratify=y)

        # concatenate our training data back together
        X = pd.concat([x_treino, y_treino], axis=1)

        # separate minority and majority classes
        not_ordem = X[X['LocalMax']==0].copy()
        ordem = X[X['LocalMax']==1].copy()

        # upsample minority
        ordem_upsampled = resample(ordem,
                                replace=True, # sample with replacement
                                n_samples=len(not_ordem), # match number in majority class
                                random_state=378) # reproducible results

        # combine majority and upsampled minority
        upsampled = pd.concat([not_ordem, ordem_upsampled])

        x_treino = upsampled[[c for c in df_slice.columns if c not in cols_rem]]
        y_treino = upsampled['LocalMax']

        display(y_treino.value_counts())

        #xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino), (x_teste, y_teste)], eval_metric=f1_score)
        param = {'max_depth':50, 'eta':1, 'objective':'binary:logistic', 'min_child_weight': 1, 'lambda': 1, 'alpha': 0, 'gamma': 0}

        df_train = xgboost.DMatrix(data=x_treino, label=y_treino)
        df_test = xgboost.DMatrix(data=x_teste, label=y_teste)

        bst = xgboost.train(param, df_train, 2, feval=f1_score)
        y_teste_pred = bst.predict(xgboost.DMatrix(data=x_teste, label=y_teste))
        y_teste_pred = [1 if i>0.5 else 0 for i in y_teste_pred]
        y_treino_pred = bst.predict(xgboost.DMatrix(data=x_treino, label=y_treino))
        y_treino_pred = [1 if i>0.5 else 0 for i in y_treino_pred]

        print(f"Treino: {accuracy_score(y_treino, y_treino_pred)}")
        print(f"Teste: {accuracy_score(y_teste, y_teste_pred)}")
        print(f"Precisão: {precision_score(y_teste, y_teste_pred)}")
        print(f"Recall: {recall_score(y_teste, y_teste_pred)}")
        print(f"F1: {f1_score(y_teste, y_teste_pred)}")
        display(confusion_matrix(y_teste, y_teste_pred, normalize='true'))
        display(confusion_matrix(y_teste, y_teste_pred,))
        
        df_training_result = df_training_result.append({
            'Removed_Cols': case,
            'Remaining_Cols': list(x.columns),
            'Local': l,
            'Train_Acc': accuracy_score(y_treino, y_treino_pred),
            'Test_Acc': accuracy_score(y_teste, y_teste_pred),
            'Precision': precision_score(y_teste, y_teste_pred),
            'Recall': recall_score(y_teste, y_teste_pred),
            'F1': f1_score(y_teste, y_teste_pred),
            'Ver_Pos': confusion_matrix(y_teste, y_teste_pred, normalize='true')[1,1]
        }, ignore_index=True)

----- LOCAL 0 -----
Index(['Local', 'UmidadeRelativa', 'PressaoAtmosferica', 'TemperaturaDoAr',
       'TemperaturaInterna', 'PontoDeOrvalho', 'RadiacaoSolar',
       'DirecaoDoVento', 'VelocidadeDoVento', 'Precipitacao', 'Prec24h',
       'Mes'],
      dtype='object')


1.0    251285
0.0    251285
Name: LocalMax, dtype: int64

Treino: 0.994436595897089
Teste: 0.9719345229962014
Precisão: 0.35028588624736684
Recall: 0.5580057526366251
F1: 0.43039378813089296


array([[0.97995227, 0.02004773],
       [0.44199425, 0.55800575]])

array([[105534,   2159],
       [   922,   1164]], dtype=int64)

...

----- LOCAL 5 -----


1.0    50324
0.0    50324
Name: LocalMax, dtype: int64

Treino: 0.9409526269771878
Teste: 0.938057934049918
Precisão: 0.17262306136210384
Recall: 0.6580976863753213
F1: 0.2735042735042735


array([[0.94310753, 0.05689247],
       [0.34190231, 0.65809769]])

array([[20340,  1227],
       [  133,   256]], dtype=int64)

In [99]:
# pd.DataFrame(
#     columns = ['Case', 'Local', 'Train_Acc', 'Test_Acc', 'Precision', 'Recall', 'F1', 'Ver_Pos']
# ).append({
#     'Case': case,
#     'Local': l,
#     'Train_Acc': accuracy_score(y_treino, y_treino_pred),
#     'Test_Acc': accuracy_score(y_teste, y_teste_pred),
#     'Precision': precision_score(y_teste, y_teste_pred),
#     'Recall': recall_score(y_teste, y_teste_pred),
#     'F1': f1_score(y_teste, y_teste_pred),
#     'Ver_Pos': confusion_matrix(y_teste, y_teste_pred, normalize='true')[1,1]
# }, ignore_index=True)
#{'Camilopolis': 1, 'Erasmo': 2, 'Paraiso': 3, 'RM': 4, 'Vitoria': 5}
display(df_training_result.tail(30))
#df_training_result.to_csv('training_test.csv', sep=';')

,Removed_Cols,Remaining_Cols,Local,Train_Acc,Test_Acc,Precision,Recall,F1,Ver_Pos
36,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...","[Local, TemperaturaDoAr, Precipitacao, Prec24h...",0,0.988332,0.964356,0.296684,0.639022,0.405229,0.639022
37,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...","[Local, TemperaturaDoAr, Precipitacao, Prec24h...",1,0.987435,0.960421,0.297787,0.633833,0.405202,0.633833
38,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...","[Local, TemperaturaDoAr, Precipitacao, Prec24h...",2,0.990866,0.970031,0.323925,0.608586,0.422807,0.608586
39,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...","[Local, TemperaturaDoAr, Precipitacao, Prec24h...",3,0.988457,0.963381,0.275978,0.612903,0.380586,0.612903
40,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...","[Local, TemperaturaDoAr, Precipitacao, Prec24h...",4,0.989178,0.965066,0.302353,0.596288,0.401249,0.596288
41,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...","[Local, TemperaturaDoAr, Precipitacao, Prec24h...",5,0.990919,0.965385,0.270136,0.560411,0.364548,0.560411
42,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...","[Local, Precipitacao, Prec24h, Mes]",0,0.956281,0.953042,0.238363,0.670182,0.351654,0.670182
43,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...","[Local, Precipitacao, Prec24h, Mes]",1,0.954656,0.943615,0.218408,0.640257,0.325708,0.640257
44,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...","[Local, Precipitacao, Prec24h, Mes]",2,0.966595,0.956595,0.242844,0.664141,0.355646,0.664141
45,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...","[Local, Precipitacao, Prec24h, Mes]",3,0.953173,0.951175,0.219144,0.647643,0.327478,0.647643


In [20]:
df_slice.drop(columns=['Data', 'Hora'])#.to_csv('dados_treinamento_15min.csv')
# [['Local', 'UmidadeRelativa', 'PressaoAtmosferica', 'TemperaturaDoAr', 'TemperaturaInterna',
#           'PontoDeOrvalho', 'RadiacaoSolar', 'DirecaoDoVento', 'VelocidadeDoVento', 'Precipitacao'
#           'PrecSum', 'LocalMax']]

,Data_Hora,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,LocalMax,PrecSum
0,2011-01-01 00:00:00,1,84.8,920.6,20.2,22.8,17.5,0.0,137.0,1.5,0.0,0.0,0.6
1,2011-01-01 00:15:00,1,84.1,920.3,20.1,22.8,17.4,0.0,76.0,1.9,0.0,0.0,0.6
2,2011-01-01 00:30:00,1,83.9,920.2,20.1,22.9,17.2,0.0,92.0,2.9,0.0,0.0,0.6
3,2011-01-01 00:45:00,1,83.5,919.9,20.2,22.9,17.3,0.0,133.0,1.4,0.0,0.0,0.6
4,2011-01-01 01:00:00,1,84.0,919.9,20.1,22.8,17.3,0.0,151.0,1.8,0.0,0.0,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463565,2019-06-30 22:45:00,5,32.6,925.9,21.7,27.9,5.2,0.0,0.0,0.0,0.0,0.0,0.0
1463566,2019-06-30 23:00:00,5,34.2,925.9,21.3,27.9,5.5,0.0,0.0,0.0,0.0,0.0,0.0
1463567,2019-06-30 23:15:00,5,35.2,925.9,21.0,27.7,5.6,0.0,0.0,0.0,0.0,0.0,0.0
1463568,2019-06-30 23:30:00,5,36.1,925.8,20.8,27.6,5.9,0.0,0.0,0.0,0.0,0.0,0.0
